In [1]:
import os

In [2]:
import pandas as pd
import numpy as np

np.set_printoptions(precision=3, suppress=True)

pd.options.display.float_format = lambda num: f'{num:.4f}'

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(rc={"figure.figsize": (12, 8)})

In [4]:
from utils.ingest import read_data
from fixed_effects import within_group
from gmm import lagged_gmm

In [5]:
from utils.printing import np_to_pmatrix

#### Constants

In [6]:
data_path = "data/hw3.xls"

### Data inject

In [7]:
data = read_data(data_path)

### 2. Homogenous panel

#### (a)

The panel is:
- homogenous
- no heterogeneity in the slope parameters
- no endogeneity
- no cross-sectional dependence

Given the assumptions, the suitable estimator is the Withing Group (or Fixed Effects) estimators.

In [8]:
regs = ["d(lnY)", "INF"]
beta, fixed_effects, resid, var, dw = within_group(data, "S/Y", regs, lags = 0)


Withing regression:  

--- β:

d(lnY)   0.293056
INF      0.179817, 


--- Standard errors:

         d(lnY)      INF
d(lnY) 0.003791 0.000090
INF    0.000090 0.001005
        
--- Durbin-Watson
1.8444  

--- Pesaran
0.0722  p=0.4712



#### 3. Dynamic homogenous panels

Within regressor with lagged value

In [9]:
regs = ["d(lnY)", "INF"]
beta, fixed_effects, resid, var, dw = within_group(data, "S/Y", regs, lags = 1, title="Lagged Within regression", het_robust=True)


Lagged Within regression:  

--- β:

d(lnY)      0.023697
INF         0.021070
lag_1_S/Y   0.848010, 


--- Standard errors:

             d(lnY)       INF  lag_1_S/Y
d(lnY)     0.001172  0.000145  -0.000295
INF        0.000005  0.000192  -0.000147
lag_1_S/Y -0.000386 -0.000110   0.000928
        
--- Durbin-Watson
1.7572  

--- Pesaran
0.0230  p=0.4908



In [10]:
_ = within_group(data, "S/Y", ["d(lnY)", "INF"], lags = 0, title="Within regression")
_ = within_group(data, "S/Y", ["d(lnY)", "INF"], lags = 0, title="Within regression", cross_sec=True)



Within regression:  

--- β:

d(lnY)   0.293056
INF      0.179817, 


--- Standard errors:

         d(lnY)      INF
d(lnY) 0.003791 0.000090
INF    0.000090 0.001005
        
--- Durbin-Watson
1.8444  

--- Pesaran
0.0722  p=0.4712


Within regression:  

--- β:

d(lnY)   0.293056
INF      0.179817, 


--- Standard errors:

          d(lnY)       INF
d(lnY)  0.006972 -0.000141
INF    -0.000141  0.002189
        
--- Durbin-Watson
1.8444  

--- Pesaran
0.0722  p=0.4712



In [11]:
with open("data/cv_matrix.tex", "w") as file:
    
    file.write(np_to_pmatrix(beta, prec = 4))

#### 4. Enogeneity in homogenous dynamic panel

In [12]:
lags = 2


_ = lagged_gmm(
    data, "S/Y",
    regressors=["d(lnY)", "INF"], lag_inst=lags, title="IV estimation of SG/Y -> S/Y"
)

_ = lagged_gmm(
    data, "S/Y",
    regressors=["d(lnY)", "INF"], lag_inst=lags, title="GMM estimation of SG/Y -> S/Y",
    gmm=True,
    is_lagged_instrumented=True
)



IV estimation of SG/Y -> S/Y:  

--- β:

d(lnY)      -1.331277
INF         -0.459295
lag_1_S/Y    3.493394, 


--- Standard errors:

              d(lnY)       INF  lag_1_S/Y
d(lnY)      5.459203  2.315545 -13.229239
INF         2.315545  1.009244  -5.721096
lag_1_S/Y -13.229239 -5.721096  32.702864
        
--- Durbin-Watson
1.8486  


GMM estimation of SG/Y -> S/Y:  

--- β:

d(lnY)      -0.213766
INF          0.008302
lag_1_S/Y    0.861791, 


--- Standard errors:

             d(lnY)       INF  lag_1_S/Y
d(lnY)     0.006201  0.000087  -0.002769
INF        0.000087  0.000390  -0.000153
lag_1_S/Y -0.002769 -0.000153   0.002973
        
--- Overidentifying Restrictions
0.0389  p=0.9980

--- Hausman
23.3514  p=0.0000

--- Durbin-Watson
1.7365  



In [13]:
lags = 2


_ = lagged_gmm(
    data, "S/Y",
    regressors=["SG/Y"], lag_inst=lags, title="IV estimation of SG/Y -> S/Y"
)

_ = lagged_gmm(
    data, "S/Y",
    regressors=["SG/Y"], lag_inst=lags, title="GMM estimation of SG/Y -> S/Y",
    gmm=True,
    is_lagged_instrumented=True
)



IV estimation of SG/Y -> S/Y:  

--- β:

SG/Y        -0.190944
lag_1_S/Y    0.541782, 


--- Standard errors:

              SG/Y  lag_1_S/Y
SG/Y      0.004458   0.012474
lag_1_S/Y 0.012474   0.042449
        
--- Durbin-Watson
1.7009  


GMM estimation of SG/Y -> S/Y:  

--- β:

SG/Y        -0.122965
lag_1_S/Y    0.802800, 


--- Standard errors:

              SG/Y  lag_1_S/Y
SG/Y      0.000629   0.000337
lag_1_S/Y 0.000337   0.001701
        
--- Overidentifying Restrictions
0.0024  p=0.9988

--- Hausman
34.5922  p=0.0000

--- Durbin-Watson
1.7923  



#### 5. Cross-sectional dependence in a homogeneous dynamic panel.

In [14]:
from ccep import ccep

In [15]:
beta, resid_by_n, cov = ccep(
        data, "S/Y", ["d(lnY)", "INF"],  title="CCEP estimator")


CCEP estimator:  

--- β:

d(lnY)      0.112180
INF         0.050982
lag_1_S/Y   0.733041, 


--- Standard errors:

             d(lnY)       INF  lag_1_S/Y
d(lnY)     0.047324  0.006842  -0.007022
INF        0.006842  0.018163  -0.003740
lag_1_S/Y -0.007022 -0.003740   0.027735
        


#### 6. Endogeneity

In [16]:
_ = lagged_gmm(
        data, "S/Y",
        regressors=["SG/Y"], lag_inst=2, title="IV estimation of SG/Y -> S/Y"
    )

_ = lagged_gmm(
        data, "S/Y",
        regressors=["SG/Y"], lag_inst=2, title="GMM estimation of SG/Y -> S/Y",
        gmm=True, is_lagged_instrumented=True
    )


IV estimation of SG/Y -> S/Y:  

--- β:

SG/Y        -0.190944
lag_1_S/Y    0.541782, 


--- Standard errors:

              SG/Y  lag_1_S/Y
SG/Y      0.004458   0.012474
lag_1_S/Y 0.012474   0.042449
        
--- Durbin-Watson
1.7009  


GMM estimation of SG/Y -> S/Y:  

--- β:

SG/Y        -0.122965
lag_1_S/Y    0.802800, 


--- Standard errors:

              SG/Y  lag_1_S/Y
SG/Y      0.000629   0.000337
lag_1_S/Y 0.000337   0.001701
        
--- Overidentifying Restrictions
0.0024  p=0.9988

--- Hausman
34.5922  p=0.0000

--- Durbin-Watson
1.7923  



#### 7. Heterogenous dynamic panel

In [17]:
from mean_group import mean_group

In [18]:
_ = mean_group(data, "S/Y", ["U"], lags=1, title="MG Estimator", verbose=1)

_ = mean_group(data, "S/Y", ["U"], lags=1, title="CCEMG Estimator", cc=True, verbose=1)


MG Estimator:  

--- β:

U           0.004935
lag_1_S/Y   0.040722, 


--- Standard errors:

                  U  lag_1_S/Y
U          0.000004  -0.000001
lag_1_S/Y -0.000001   0.000005
        
--- Long run effect U -> S/Y
0.0409  


CCEMG Estimator:  

--- β:

U           -0.936113
lag_1_S/Y    0.566169, 


--- Standard errors:

                  U  lag_1_S/Y
U          0.893532  -0.510421
lag_1_S/Y -0.510421   0.292084
        
--- Long run effect U -> S/Y
0.0221  

